# Big Data Analytics with MovieLens Dataset

In this Jupyter Notebook, we will use the [MovieLens 20M Dataset](https://grouplens.org/datasets/movielens/20m/) on movie ratings to answer several tasks by using `pySpark`. The exercises are structured as a guideline to get familiar with the Spark and pyspark syntax. Have also a look on the [official pySpark documentation](https://spark.apache.org/docs/latest/api/python/pyspark.html). 

**Introduction to Movielens dataset**

The Introduction exercises have the following goals:
- Reading and understanding the schema of our movielens dataset
- Calculating some summary statistics of our dataset
- Learn how to perform joins and aggregations using Spark
- Sample exercise 1: Which movies are the most popular ones?
- Sample exercise 2: Creating RDD's


**Exercises**
- Basic exercise 1: Which movies have the highest number of ratings?
- Basic exercise 2: Something with RDD's
- Advanced exercise 1: Which movies are a matter of taste?

**Remark**: Pyspark needs to be installed to work with the Jupyter Notebook (e.g. Anaconda3)



## Initialize Sparksession

Execute the following cell to initialize a Sparksession:

In [27]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('MovieLens').getOrCreate()

## Introduction: Reading the dataset

Our movielens dataset contains 20 million ratings and 465'000 tag applications applied to 27'000 movies by 138'000 users. It also includs tag genome data with 12 million relevance scores across 1100 tags.

The whole dataset contains six CSV files:
- genome-scores.csv
- genome-tags.csv
- links.csv
- movies.csv
- ratings.csv
- tags.csv

First, we will have a look on the **`movies`** and **`ratings`** dataframes.

To read a CSV file in our "ml-20m" folder, we access the `DataFrameReader` class through `read` and call the `csv()` method on it. We also specify `option("header", "true")` since the first row of the file contains our column names. 

In [28]:
ratings = spark.read.option("header", "true").csv("ml-20m/ratings.csv")
ratings.show(20)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|      2|   3.5|1112486027|
|     1|     29|   3.5|1112484676|
|     1|     32|   3.5|1112484819|
|     1|     47|   3.5|1112484727|
|     1|     50|   3.5|1112484580|
|     1|    112|   3.5|1094785740|
|     1|    151|   4.0|1094785734|
|     1|    223|   4.0|1112485573|
|     1|    253|   4.0|1112484940|
|     1|    260|   4.0|1112484826|
|     1|    293|   4.0|1112484703|
|     1|    296|   4.0|1112484767|
|     1|    318|   4.0|1112484798|
|     1|    337|   3.5|1094785709|
|     1|    367|   3.5|1112485980|
|     1|    541|   4.0|1112484603|
|     1|    589|   3.5|1112485557|
|     1|    593|   3.5|1112484661|
|     1|    653|   3.0|1094785691|
|     1|    919|   3.5|1094785621|
+------+-------+------+----------+
only showing top 20 rows



Each tuple of the `ratings` DataFrame represents one rating (`rating`) for one movie (`movieId`) by one user (`userId`). The ratings ranges from 0.5 stars (worst) up to 5.0 stars (best). 

We can also have look on the Schema of our dataset (column names and types) by using the `printSchema()` method.

In [29]:
ratings.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



Do the same for the `movies.csv` file. What kind of data is available and how does the schema look like?

In [30]:
movies = spark.read.option("header", "true").csv("ml-20m/movies.csv")
movies.show(20)

movies.printSchema()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

## Intro exercise 1: Which movies are the most popular ones?

To get the most popular movies, we are looking for the movies with the highest number of ratings. In this task, we assume the number of ratings as a representative for the number of views. To do this, we will perform the following *transformations* on the `ratings` DataFrame: 
- group by `movieId`
- count the number of users (`userId`) associated with each movie 
- rename this column to `num_ratings`
- sort by `num_ratings` in descending order 

We do these transformations in `pySpark` and store the DataFrame as `most_popular`. Have also a look on the [official pySpark documentation](https://spark.apache.org/docs/latest/api/python/pyspark.html).

**HINT**:
- Use `agg(count())` to perform an aggregate calculation on grouped data. 
- Don't forget that transformations are [lazy](https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-operations) in spark. We need to call an action (e.g. `show()`, `take()`) explicitly to see the results. 

In [31]:
from pyspark.sql.functions import *

most_popular = ratings.groupBy("movieId").agg(count("userId")).withColumnRenamed("count(userId)", "num_ratings").sort(desc("num_ratings"))

most_popular.show(20)

+-------+-----------+
|movieId|num_ratings|
+-------+-----------+
|    296|      67310|
|    356|      66172|
|    318|      63366|
|    593|      63299|
|    480|      59715|
|    260|      54502|
|    110|      53769|
|    589|      52244|
|   2571|      51334|
|    527|      50054|
|      1|      49695|
|    457|      49581|
|    150|      47777|
|    780|      47048|
|     50|      47006|
|   1210|      46839|
|    592|      46054|
|   1196|      45313|
|   2858|      44987|
|     32|      44980|
+-------+-----------+
only showing top 20 rows



Unfortunately, the resulting table only contains `movieId` and `num_ratings`. The title of the movie is stored in the `movies` DataFrame. So, we need an inner join of our `most_popular` DataFrame with the `movies` DataFrame on `movieId`.

In [32]:
most_popular_movies = most_popular.join(movies, most_popular.movieId == movies.movieId)
most_popular_movies.show(20, truncate=False)

+-------+-----------+-------+-----------------------------------------------------+-------------------------------------------+
|movieId|num_ratings|movieId|title                                                |genres                                     |
+-------+-----------+-------+-----------------------------------------------------+-------------------------------------------+
|296    |67310      |296    |Pulp Fiction (1994)                                  |Comedy|Crime|Drama|Thriller                |
|356    |66172      |356    |Forrest Gump (1994)                                  |Comedy|Drama|Romance|War                   |
|318    |63366      |318    |Shawshank Redemption, The (1994)                     |Crime|Drama                                |
|593    |63299      |593    |Silence of the Lambs, The (1991)                     |Crime|Horror|Thriller                      |
|480    |59715      |480    |Jurassic Park (1993)                                 |Action|Adventure|Sci-

We now have a list of the most popular (or most rated) movies of our movielens dataset. 

## Intro exercise 2: Understanding RDD's (Resilient Distributed Datasets)

--> still ongoing

In [10]:
# Load the data into RDD
data = sc.textFile("ml-20m/ratings.csv")

# Split the RDD 
ratings = data.map(lambda l: l.split(','))
ratings.take(5)

[['userId', 'movieId', 'rating', 'timestamp'],
 ['1', '2', '3.5', '1112486027'],
 ['1', '29', '3.5', '1112484676'],
 ['1', '32', '3.5', '1112484819'],
 ['1', '47', '3.5', '1112484727']]

In [12]:
# map
Reversed = ratings.map(lambda rating: rating[::-1])
Reversed.take(5)

[['timestamp', 'rating', 'movieId', 'userId'],
 ['1112486027', '3.5', '2', '1'],
 ['1112484676', '3.5', '29', '1'],
 ['1112484819', '3.5', '32', '1'],
 ['1112484727', '3.5', '47', '1']]

In [18]:
# flatMap
characters = ratings.flatMap(lambda rating: list(rating))
characters.take(10)

['userId',
 'movieId',
 'rating',
 'timestamp',
 '1',
 '2',
 '3.5',
 '1112486027',
 '1',
 '29']

## Basic exercise 1: Which movies have the highest ratings (in average)?

Now we want to see which movies are rated to be the best. We will use the `ratings` DataFrame and: 

- Group by `movieId` 
- Calculate the average rating for each movie and rename this column to `avg_rating`
- Sort by `avg_rating` in descending order 
- Join the resulting DataFrame with the `movies` DataFrame to get the movienames.

**NOTE** Be sure that you read the movies file in the first part

In [33]:
top_rated = ratings.groupBy("movieId").agg(avg(col("rating"))).withColumnRenamed("avg(rating)", "avg_rating").sort(desc("avg_rating"))

top_rated_movies = top_rated.join(movies, top_rated.movieId == movies.movieId)

top_rated_movies.show(20)

+-------+----------+-------+--------------------+--------------------+
|movieId|avg_rating|movieId|               title|              genres|
+-------+----------+-------+--------------------+--------------------+
|  95517|       5.0|  95517|Barchester Chroni...|               Drama|
| 109715|       5.0| 109715|Inquire Within (2...|              Comedy|
| 111548|       5.0| 111548|Welcome to Austra...|         Documentary|
| 129905|       5.0| 129905|The Floating Cast...|        Comedy|Drama|
|  98761|       5.0|  98761|Shaolin Temple 2:...|       Action|Comedy|
|  27914|       5.0|  27914|Hijacking Catastr...|         Documentary|
| 129305|       5.0| 129305|Pretty Things (2001)|               Drama|
| 106113|       5.0| 106113|Such Hawks Such H...|         Documentary|
|  94431|       5.0|  94431|Ella Lola, a la T...|  (no genres listed)|
|  93707|       5.0|  93707|Prom Queen: The M...|        Comedy|Drama|
| 127256|       5.0| 127256|  The Old Gun (1975)|  Drama|Thriller|War|
|  959

The resulting Dataframe is maybe not meaningful. We should also consider the number of ratings by doing an aggregation `agg()` call. 

In [34]:
top_rated = ratings.groupBy("movieId").agg(count("userId"), avg(col("rating"))).withColumnRenamed("count(userId)", "num_ratings").withColumnRenamed("avg(rating)", "avg_rating")

top_rated_movies = top_rated.join(movies, top_rated.movieId == movies.movieId).sort(desc("avg_rating"), desc("num_ratings"))

top_rated_movies.show(20)

+-------+-----------+----------+-------+--------------------+-------------------+
|movieId|num_ratings|avg_rating|movieId|               title|             genres|
+-------+-----------+----------+-------+--------------------+-------------------+
| 108527|          2|       5.0| 108527|  Catastroika (2012)|        Documentary|
| 103871|          2|       5.0| 103871|Consuming Kids: T...|        Documentary|
|  86055|          1|       5.0|  86055|Foster Brothers, ...|             Comedy|
| 109253|          1|       5.0| 109253|Argentina latente...|        Documentary|
| 117606|          1|       5.0| 117606|      Divorce (1945)|              Drama|
| 129305|          1|       5.0| 129305|Pretty Things (2001)|              Drama|
| 113860|          1|       5.0| 113860|Codes of Gender, ...|        Documentary|
| 106082|          1|       5.0| 106082|Shock and Awe: Th...|        Documentary|
| 113790|          1|       5.0| 113790|Peace, Propaganda...|        Documentary|
|  98761|       

All of the movies with `avg_rating` of exactly 5.0 have 2 or less `num_ratings` . We must investigate the distribution of `num_ratings` to only consider movies that have a minimum number of ratings. Let's calculate some summary statistics within Spark.

In [35]:
# Calculate average, minimum, and maximum of num_ratings
top_rated_movies.select([mean('num_ratings'), min('num_ratings'), max('num_ratings')]).show(1)

+-----------------+----------------+----------------+
| avg(num_ratings)|min(num_ratings)|max(num_ratings)|
+-----------------+----------------+----------------+
|747.8411232425965|               1|           67310|
+-----------------+----------------+----------------+



In [39]:
top_rated_movies.where("num_ratings > 800").show(20, truncate=False)

+-------+-----------+------------------+-------+---------------------------------------------------------------------------+-----------------------------------------+
|movieId|num_ratings|avg_rating        |movieId|title                                                                      |genres                                   |
+-------+-----------+------------------+-------+---------------------------------------------------------------------------+-----------------------------------------+
|318    |63366      |4.446990499637029 |318    |Shawshank Redemption, The (1994)                                           |Crime|Drama                              |
|858    |41355      |4.364732196832306 |858    |Godfather, The (1972)                                                      |Crime|Drama                              |
|50     |47006      |4.334372207803259 |50     |Usual Suspects, The (1995)                                                 |Crime|Mystery|Thriller                   

**Testing SparkSQL**

In [40]:
spark.sql("select * from top_rated_movies where num_ratings > 800").show(3, truncate = False)

AnalysisException: 'Table or view not found: top_rated_movies; line 1 pos 14'

## Basic exercise 2: Something with RDD's

## Advanced exercise 1: Which movies are a matter of taste?

As you know, movies are a matter of taste. There are for sure some movies, which you would rate with a 5 whereas your friend rates the same movie with a 2. These are the movies that divide your opinon. Try to find out, which movies belong to this category.

**HINT**

- We need to consider the standard deviation of the movie ratings
- Also, try to consider only movies that have some minimum number of ratings (e.g. 700) 
- Join with the movies table to get the movie names

In [44]:
ratings_stddev = ratings.groupBy("movieId").agg(count("userId").alias("num_ratings"), 
avg(col("rating")).alias("avg_rating"),stddev(col("rating")).alias("std_rating")).where("num_ratings > 700")

ratings_stddev.show(10)

+-------+-----------+------------------+------------------+
|movieId|num_ratings|        avg_rating|        std_rating|
+-------+-----------+------------------+------------------+
|    296|      67310| 4.174231169217055|0.9760762295742448|
|   1090|      15808| 3.919977226720648|0.8272067263021853|
|   3959|       2869| 3.699372603694667|0.8607671626686736|
|   2294|      10163| 3.303207714257601|0.9047000233824075|
|   6731|       1173|3.5571184995737424| 0.918929235043451|
|  48738|       4163| 3.895868364160461|0.6545425744035035|
|   3210|       7968|3.6711219879518073|0.9119945444634687|
|  88140|       2313|3.5536100302637266|0.8756738276071163|
|    467|        741|3.3832658569500675|1.2721828955638057|
|   2088|       3539| 2.562729584628426|1.0602369201260298|
+-------+-----------+------------------+------------------+
only showing top 10 rows



In [46]:
matterofTaste_movies = ratings_stddev.join(movies, ratings_stddev.movieId == movies.movieId)

matterofTaste_movies.sort(desc("std_rating")).show(10, truncate=False)

+-------+-----------+------------------+------------------+-------+---------------------------------------------+-------------------------------------+
|movieId|num_ratings|avg_rating        |std_rating        |movieId|title                                        |genres                               |
+-------+-----------+------------------+------------------+-------+---------------------------------------------+-------------------------------------+
|1924   |2304       |2.6319444444444446|1.420171182322382 |1924   |Plan 9 from Outer Space (1959)               |Horror|Sci-Fi                        |
|4255   |1550       |2.1351612903225807|1.3504497277537106|4255   |Freddy Got Fingered (2001)                   |Comedy                               |
|78772  |884        |2.670814479638009 |1.3485057430514158|78772  |Twilight Saga: Eclipse, The (2010)           |Fantasy|Romance|Thriller|IMAX        |
|72407  |1135       |2.565638766519824 |1.3367548401080391|72407  |Twilight Saga: New Mo

Without any surprise, Twilight is a highly debated movie ;) 